# Lab 4: Problem set on Newton-Cotes formulas

In [ ]:
from IPython.core.display import HTML
def css_styling():
    styles = open("../styles/tma4215.css", "r").read()
    return HTML(styles)

# Comment out next line and execute this cell to restore the default notebook style 
css_styling()


Inserting my secret latex macros here ...
$$
\DeclareMathOperator{\Div}{div}
\DeclareMathOperator{\Grad}{grad}
\DeclareMathOperator{\Curl}{curl}
\DeclareMathOperator{\Rot}{rot}
\DeclareMathOperator{\ord}{ord}
\DeclareMathOperator{\Kern}{ker}
\DeclareMathOperator{\Image}{im}
\DeclareMathOperator{\spann}{span}
\DeclareMathOperator{\rank}{rank}
\DeclareMathOperator{\dist}{dist}
\DeclareMathOperator{\diam}{diam}
\DeclareMathOperator{\sig}{sig}
\DeclareMathOperator{\Id}{Id}
\DeclareMathOperator{\CQR}{CQR}
\DeclareMathOperator{\QR}{QR}
\DeclareMathOperator{\TR}{TR}
\DeclareMathOperator{\CTR}{CTR}
\DeclareMathOperator{\SR}{SR}
\DeclareMathOperator{\CSR}{CSR}
\DeclareMathOperator{\NCR}{NCR}
\DeclareMathOperator{\MR}{MR}
\newcommand{\RR}{\mathbb{R}}
\newcommand{\NN}{\mathbb{N}}
\newcommand{\VV}{\mathbb{V}}
\newcommand{\dGamma}{\,\mathrm{d} \Gamma}
\newcommand{\dGammah}{\,\mathrm{d} \Gamma_h}
\newcommand{\dx}{\,\mathrm{d}x}
\newcommand{\dy}{\,\mathrm{d}y}
\newcommand{\ds}{\,\mathrm{d}s}
\newcommand{\dt}{\,\mathrm{d}t}
\newcommand{\dS}{\,\mathrm{d}S}
\newcommand{\dV}{\,\mathrm{d}V}
\newcommand{\dX}{\,\mathrm{d}X}
\newcommand{\dY}{\,\mathrm{d}Y}
\newcommand{\dE}{\,\mathrm{d}E}
\newcommand{\dK}{\,\mathrm{d}K}
\newcommand{\dM}{\,\mathrm{d}M}
\newcommand{\cd}{\mathrm{cd}}
\newcommand{\onehalf}{\frac{1}{2}}
\newcommand{\bfP}{\boldsymbol P}
\newcommand{\bfx}{\boldsymbol x}
\newcommand{\bfy}{\boldsymbol y}
\newcommand{\bfa}{\boldsymbol a}
\newcommand{\bfu}{\boldsymbol u}
\newcommand{\bfv}{\boldsymbol v}
\newcommand{\bfe}{\boldsymbol e}
\newcommand{\bfb}{\boldsymbol b}
\newcommand{\bfc}{\boldsymbol c}
\newcommand{\bfq}{\boldsymbol q}
\newcommand{\bfy}{\boldsymbol y}
\newcommand{\bff}{\boldsymbol f}
\newcommand{\bfp}{\boldsymbol p}
\newcommand{\bft}{\boldsymbol t}
\newcommand{\bfj}{\boldsymbol j}
\newcommand{\bfB}{\boldsymbol B}
\newcommand{\bfV}{\boldsymbol V}
\newcommand{\bfE}{\boldsymbol E}
\newcommand{\bfB}{\boldsymbol B}
\newcommand{\bfzero}{\boldsymbol 0}
$$

The main task of this problem set is the following: for a given interval $(a,b)$ and $n+1$ equally distributed nodes $x_i = a + i \tfrac{(b-a)}{n}$
for $i=0,\ldots n$, tabulate the weights for the Newton-Cotes formula up to $n=14$.
The problem boils down to 2 task, namely 

1. Defining the Lagrange polynomials $L_{in}$ for $i=0, \ldots, n$
2. Computing the weights $w_i = \int_a^b L_{in}(x) \dx$

__Before you start:__ For the implementation of the task, we recommend to use the [sympy](https://docs.sympy.org/latest/index.html#) python module  for symbolic mathematics to perform tasks such as (symbolic) integration.
Spend some time to browse  through the  [sympy tutorial](https://docs.sympy.org/latest/tutorial/index.html) and the [help on symbolic integration](https://docs.sympy.org/latest/modules/integrals/integrals.html). Using ```sympy```__Before you start:__ For the implementation of the task, we recommend to use the [sympy](https://docs.sympy.org/latest/index.html#) python module  for symbolic mathematics to perform tasks such as (symbolic) integration.
Spend some time to browse  through the  [sympy tutorial](https://docs.sympy.org/latest/tutorial/index.html) and the [help on symbolic integration](https://docs.sympy.org/latest/modules/integrals/integrals.html). Using ```sympy``` you can for instance do something like this for $n=2$:


In [ ]:
# import symbol x from sympy so that you can define symbolic functions of x
from sympy.abc import x
# import symbolic integration
from sympy import integrate 
import numpy as np
from functools import reduce
from sympy.utilities.lambdify import lambdify
import matplotlib.pyplot as plt


# Define
a, b = 0, 1 
xqs = np.linspace(a,b,3)
# Define L_02 (not normalized)
L_02 = (x-xqs[1])*(x-xqs[2])
# Normalize it to satisfy l_02(x_0) = 1
L_02 = L_02/L_02.subs(x,xqs[0])

# Now integrate L_02 to compute the first weight
w_0 = integrate(L_02, (x, a, b))
print("w0 = {}".format(w_0))

Of course, 
since you are asked to do comupute all $n+1$ weights  $n=1,\ldots,14$,
you need to automatize the construction of the corresponding Lagrange polynoms.
So proceed as follows


__a__) Write  a python function ```lagrange_polys``` which takes a list of $n+1$ quadrature points
and returns a list of the corresponding $n+1$ Lagrange polynoms $\{L_{in}\}_{i=0}^n$
defined a symbolic function using ```sympy```:

In [ ]:
import sympy as sp

def lagrange_polys(xqs):
    n = len(xqs)
    Ls = []

    for i in range(n):
        L_i = sp.prod((x - xqs[j]) for j in range(n) if j != i)

        # normalize:
        L_i = L_i / L_i.subs(x, xqs[i])
        Ls.append(L_i)

    return Ls


__b__) Now the easy part! Employ your brand new ```def lagrange_polys``` function and implement a python function which takes as argument the desired degree of exactness $n$ and the interval end points
$a,b$ and returns a list of quadrature points $\{x_i\}_{i=0}^n$ and quadrature weights $\{w_i\}_{i=0}^n$:

In [ ]:
def newton_cotes_formula(n, a, b):
    xqs = np.linspace(a,b,n+1)
    
    Ls = lagrange_polys(xqs)
    ws =[]
    
    for i in range(n+1):
        w_i = integrate(Ls[i], (x, a, b))
        ws.append(w_i)
    return (xqs, ws)

__c__) Before you tabulate the quadrature weights with you newly implemented function, make sure that you implement them correctly. More, specifically, check for $n=1,\ldots 14$
that the computed Newton-Cotes formula integrates polynomials up to order $n$ __exactly__.

For $n$ is even, check that the corresponding Newton-Cotes rules even integrate polynomials up to order $n+1$  exactly (and not only up to $n$).

Note:  Due to floating point related errors and some numerical instabilities when computing
higher order Lagrange polynomials and integrals, the difference between the exact integral 
and the numerically error won't be 0, but around the machine precision for $n=1,2$ and then
for each increase of the order $n$ you will roughly loose of significant digit in
the difference between the two.

It might be useful to implement a little function ```qr``` first,
which takes $f$, $\{x_i\}_{i=0}^n$ and $\{w_i\}_{i=0}^n$ and
applies the corresponding quadrature to compute $\int_a^b f\dx$ numerically.


In [ ]:
def qr(f, xqs, ws):
    n = len(xqs)
    qr_f = np.sum(ws * f(xqs))
    return qr_f


def newton_cotes_rule(f_sp, a, b, n):
    """ f_sp: sympy exp"""
    f_np = lambdify(x, f_sp, "numpy")
    xqs, ws = newton_cotes_formula(n, a, b)
    return qr(f_np, xqs, ws)
    

In [ ]:
mono = lambda x,m: x**m
print(mono(x,2))

a, b = 0, 1


for n in range(1,15):
    f_sp = mono(x, n)
    res = newton_cotes_rule(f_sp, a, b, n)
    print("numeric: ", res)
    print("analytic: ", float(integrate(f_sp, (x, a, b))))
    

__d__) Tabulate the quadrature weights for the Newton-Cotes rule for $n=1,\ldots 14$. For which $n$  should you
refrain from using the resulting quadrature rule (and why?)

In [ ]:
import matplotlib.pyplot as plt

a, b = 0, 1
fig, ax = plt.subplots()
for n in range(1,15):
    xqs, ws = newton_cotes_formula(n, a, b)
    ws = [str(w)[:5] for w in ws]
    print(f"{n:2g} |", " |".join(ws))
    if n < 8:
        print(len(xqs), len(ws))
        ax.plot(xqs, ws)

plt.show()
del fig, ax


VEKTER SKAL IKKE VÆRE NEGATIVE.
FIX PLOT

__e__) Finally, since we have all the nice machinery in place, we take a little extra-tour
and investigate the convergence of the Newton-Cotes rules for $n\to \infty$.

More precisely, compute for $f(x) = \cos(x)$ the integral $\int_{-4}^{5} f(x) \dx $ first analytically 
and then numerically using the Newton-Cotes rules for $n=1,\ldots, 14$ and tabulate
the quadrature error $E_n(f) = \int_{-4}^{5} \cos(x)\dx - \NCR(\cos, n)$.

Finally, repeat the same experiment for for $f(x) = \tfrac{1}{1+x^2}$.